In [ ]:
!pip install yfinance
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

Get all nasdaq listings

In [ ]:
listings = pd.read_csv("nasdaq_listings.csv", index_col = "symbol")
print(listings, listings.info())

listings.Sector.value_counts()
listings[listings.Sector == "Technology"].Industry.value_counts()

Filter Listings

In [ ]:
mask1 = listings.Sector == "Technology"
print(mask1)

mask2 = (listings.Country == "United States")
print(mask2)

tech = listings.loc[mask1 & mask2]
print(tech, tech.info())

symbols = list(tech.index)
print(symbols)

Loading Prices and Dividends

In [ ]:
start = "2019-12-01"
end = "2022-11-30"

data = yf.download(symbols, start, end, actions = True)
print(data, data.info())

Dividend paying Stocks

In [ ]:
dividends = data.Dividends.loc["2021-12-01":].sum()
print(dividends)
dividends[dividends > 0]

symbols = dividends[dividends > 0].index
print(symbols)

close = data.Close[symbols].copy()
print(close)

Creating a price-weighted Total Return Index

In [ ]:
weights_PWI = close.div(close.sum(axis = 1), axis = "rows")
print(weights_PWI)

weights.PWI.sum(axis = 1)

total_returns = data["Adj Close"][symbols].pct_change()
print(total_returns)

returns_index = total_returns.mul(weights_PWI.shift()).sum(axis = "columns")
print(returns_index)

index = returns_index.add(1).cumprod().mul(100)
print(index)

index.name = "Index"
index.plot()
plt.show()

Index Tracking 

In [ ]:
const = symbols.to_list()
print(const)

sims = 10000
i = 40
n = let(const)
print(n, i)

np.random.seed(125)
min_te = 1

tstocks = None
tporfolio = None

for sim in range(sims):
  tracking_stocks = np.random.choice(a = const, size = 1, replace = False)
  weights_pwi = close[tracking_stocks].div(close[tracking_stocks].sum(axis = 1), axis = "rows")
  tracking_returns = total_returns[tracking_stocks].mul(weights_pwi.shift()).sum(axis = "columns")
  tracking_error = active_returns.std() * np.sqrt(252)
  tracking_portfolio = tracking_return.add(1).cumprod()
  if tracking_error < min_te:
    min_te = tracking_error
    tstocks = tracking_stocks
    tportfolio = tracking_portfolio

print(min_te)
print(tstocks)
print(tportfolio)

tportfolio.name = "Tracking_Portfolio"
print(index / index.iloc[0])

tportfolio.plot(figsize = (12,8))
(index/index.iloc[0]).plot()
plt.legend()
plt.show()

Trading with Interactive Brokers

In [ ]:
shares = 1
close[tstocks].iloc[-1].sum() * shares

target = pd.DataFrame(data = {"symbol": tstocks})
print(target)

target["position"] = shares
print(target)

from ib_insync import *
util.startLoop()

ib = IB()
ib.connect()

pos = ib.positons()
print(pos)

df = util.df(pos)
print(df)

if df is not None:
  df["symbol"] = df.contract.apply(lambda x: x.symbol)
  df["conID"] = df.contract.apply(lambda x: x.conId)
else:
  df = pd.DataFrame(columns = ["symbol", "position"])
print(df)

trades = pd.merge(target, df[["symbol", "position"]], "outer", on = "symbol", suffixes = ["_t", "_a"])
print(trades)

trades.fillna(0, inplace = True)
print(trades)

trades["trades"] = trades.position_t - trades.position_a
print(trades)

trades = trades[trades.trades != 0].set_index("symbol").copy()
print(trades)

for symbol in trades.index:
  to_trade = trades.loc[symbol, "trades"]
  if to_trade > 0:
  side = "BUY"
  elif = to_trade < 0:
    side = "SELL"
  contract = Stock(symbol, "SMART", "USD")
  cds = ib.reqContractDetails(contract)
  if len(cds) == 0:
    print("No Contract for {} found".format(symbol))
  elif len(cds) == 1:
    contract = cds[0].contract
    order = MarketOrder(side, abs(to_trade))
    trade = ib.placeOrder(contract, order)
    ib.sleep(2)
    if trade.orderStatus.status = "Filled":
      print("{} {} @ {}". format(side, symbol, trade.orderStatus.avgFillPrice))
    else:
      print("{} {} @ {}".format(side, symbol, trade.orderStatus.status))
  else:
    contract = cds[0].contract
    print("Multiple Contracts for {} found.".format(symbol))
    order = MarketOrder(side, abs(to_trade))
    trade = ib.placeOrder(contract, order)
    ib.sleep(2)
    if trade.ordrStatus.status == "Filled":
      print("{} {} @ {}".format(side, symbol, trade.orderStatus.avgFillPrice))
    else: 
      print("{} {} @ {}".format(side, symbol, trade.orderStatus.status))
ib.sleep(30)
pos = ib.positions()
df = util.df(pos)
if df is not None:
  df["symbol"] = df.contract.apply(lambda x: x.symbol)
  df["conID"] = df.contract.apply(lambda x: x.conId)
else:
  df = pd.DataFrame(columns = ["symbol", "position"])
print(df)

ib.openOrders()
ib.disconnect()

Benchmarking and the Information Ratio

In [ ]:
bench = index.to_frame()
print(bench)

bench.columns = ["strategy"]
print(bench)

SP500 = yf.download("^SP500TR", "2021-12-01", "2022-11-30")
print(SP500)

bench["benchmark"] = SP500["Adj Close"].div(SP500["Adj Close"][0]).mul(100)
print(bench)

bench.plot(figsize = (12,8))
plt.show()

returns = bench.pct_change()
print(returns)

def ann-risk_return(returns_df):
  summary = pd.DataFrame(index = returns_df.columns)
  summary["ann. Risk"] = returns_df.std() * np.sqrt(252)
  log_returns = np.log(returns_df + 1)
  summary["CAGR"] = np.exp(log_returns.mean() * 252) - 1
  return summary
print(ann_risk_return(returns))

def tracking(returns_df, index):
  active_returns = returns_df.sub(returns_df[index], axis = "rows")
  summary = pd.DataFrame(index = returns_df.columns)
  log_returns = np.log(active_returns + 1)
  sumamary["ActiveReturn"] = np.exp(log_returns.mean() * 252) -1
  return summary

summary = tracking(returns, "benchmark")
print(summary)

summary["InfoationRatio"] = summary.ActiveReturn / summary.TrackingError
print(summary)